# 01 Attribute and Pickle the Network
Step 01: Add key attributes to the 'raw' network and create pickles

In [1]:
import os
import sys
import pickle

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import Parameters
from lasso import metcouncil

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O

In [10]:
input_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'client_met_council_network', 'network_standard')
lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'lasso')
output_dir = input_dir

## Read Roadway and Transit Networks

In [6]:
link_file = os.path.join(input_dir, 'link.json')
node_file = os.path.join(input_dir, 'node.geojson')
shape_file = os.path.join(input_dir, 'shape.geojson')

roadway_net = RoadwayNetwork.read(link_file = link_file, node_file = node_file, shape_file = shape_file)

2020-10-21 13:40:00, INFO: Reading from following files:
-/Users/wsp/Documents/GitHub/client_met_council_network/network_standard/link.json
-/Users/wsp/Documents/GitHub/client_met_council_network/network_standard/node.geojson
-/Users/wsp/Documents/GitHub/client_met_council_network/network_standard/shape.geojson.
2020-10-21 13:40:00, INFO: Reading from following files:
-/Users/wsp/Documents/GitHub/client_met_council_network/network_standard/link.json
-/Users/wsp/Documents/GitHub/client_met_council_network/network_standard/node.geojson
-/Users/wsp/Documents/GitHub/client_met_council_network/network_standard/shape.geojson.
2020-10-21 13:42:01, INFO: Read 1134074 links from /Users/wsp/Documents/GitHub/client_met_council_network/network_standard/link.json
2020-10-21 13:42:01, INFO: Read 1134074 links from /Users/wsp/Documents/GitHub/client_met_council_network/network_standard/link.json
2020-10-21 13:42:01, INFO: Read 354346 nodes from /Users/wsp/Documents/GitHub/client_met_council_network/n

In [7]:
transit_net = TransitNetwork.read(feed_path = input_dir)

2020-10-21 13:46:33, INFO: Read in transit feed from: /Users/wsp/Documents/GitHub/client_met_council_network/network_standard
2020-10-21 13:46:33, INFO: Read in transit feed from: /Users/wsp/Documents/GitHub/client_met_council_network/network_standard
2020-10-21 13:46:33, INFO: Removing calendar.txt from transit network config because file not found
2020-10-21 13:46:33, INFO: Removing calendar.txt from transit network config because file not found
2020-10-21 13:46:33, INFO: Removing calendar_dates.txt from transit network config because file not found
2020-10-21 13:46:33, INFO: Removing calendar_dates.txt from transit network config because file not found
2020-10-21 13:46:34, INFO: Removing fare_attributes.txt from transit network config because file not found
2020-10-21 13:46:34, INFO: Removing fare_attributes.txt from transit network config because file not found
2020-10-21 13:46:34, INFO: Removing fare_rules.txt from transit network config because file not found
2020-10-21 13:46:34,

## Keep the Drive Links
During the network updates, this allows for faster I/O. Also remove the legacy number of lanes coding, which is now computed via the `calculate_number_of_lanes` method.

In [14]:
roadway_net.links_df = roadway_net.links_df[roadway_net.links_df['drive_access'] == 1]
roadway_net.links_df = roadway_net.links_df.drop('lanes', axis = 1)

## Attribute the Network

In [11]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-10-21 13:47:06, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-10-21 13:47:06, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso


In [15]:
r_net = metcouncil.calculate_number_of_lanes(
        roadway_net=roadway_net,
        parameters=parameters,
)
r_net.links_df.lanes.value_counts()

2020-10-21 13:49:03, INFO: Calculating Number of Lanes as network variable: 'lanes'
2020-10-21 13:49:03, INFO: Calculating Number of Lanes as network variable: 'lanes'
2020-10-21 13:49:03, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-10-21 13:49:03, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-10-21 13:49:03, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2020-10-21 13:49:03, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2020-10-21 13:49:23, INFO: Finished calculating number of lanes to: lanes
2020-10-21 13:49:23, INFO: Finished calculating number of lanes to: lanes


1    325181
2     26560
3      1734
4       545
5        66
6         6
Name: lanes, dtype: int64

In [16]:
r_net = metcouncil.calculate_assign_group_and_roadway_class(
        roadway_net=r_net,
        parameters=parameters,
)
r_net.links_df.assign_group.value_counts()

2020-10-21 13:49:29, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2020-10-21 13:49:29, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2020-10-21 13:49:29, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2020-10-21 13:49:29, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2020-10-21 13:50:21, INFO: Finished calculating assignment group variable assign_group and roadway class variable roadway_class
2020-10-21 13:50:21, INFO: Finished calculating assignment group variable assign_group and roadway class variable roadway_class


50    223143
7      54336
6      44241
9      24134
4       4384
2       1404
1       1388
15       937
11       123
8          2
Name: assign_group, dtype: int64

## Create a Scenario

In [17]:
base_scenario = {"road_net": r_net, "transit_net": transit_net}

In [18]:
working_scenario = Scenario.create_scenario(base_scenario = base_scenario)

2020-10-21 13:50:33, INFO: Creating Scenario
2020-10-21 13:50:33, INFO: Creating Scenario


## Make Pickles

In [19]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_00.pickle')
pickle.dump(working_scenario, open(working_scenario_filename, 'wb'))